
**Nome:**Laura\
**Cognome**: Nembrini \
**Matricola:** 819059

# ASSIGNMENT 5


In [1]:
from tensorflow import keras
keras.__version__

'2.7.0'

In [2]:
import tarfile
from time import time
from shutil import copyfile

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
t0 = time()
# Link Google Drive account, transfer dataset, and extract files
copyfile('gdrive/My Drive/101_ObjectCategories_split.tar', '101_ObjectCategories_split.tar')
tar = tarfile.open('101_ObjectCategories_split.tar')
tar.extractall()
tar.close()
print("File transfer completed in %0.3f seconds" % (time() - t0))

File transfer completed in 4.682 seconds


In [5]:
base_path = '101_ObjectCategories_split/'

# Data augmentation

Sono state applicate delle trasformazioni di preprocessing e data augmentation. In particolare, nella fase di data augmentation, vengono effetuate le seguenti operazioni:

+ rotation range pari a 20
+ width_shift_range pari a 0.2
+ height_shift_range pari a 0.2
+ zoom_range pari a 0.2

In [6]:
train_processing =  keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = .2,
    height_shift_range = .2,
    zoom_range = .2)


train_generator = train_processing.flow_from_directory('101_ObjectCategories_split/train', 
                                                       target_size = (224, 224),
                                                       color_mode="rgb",
                                                       batch_size = 32,
                                                       seed = 1,
                                                       class_mode= 'categorical',
                                                       shuffle = True)

Found 4600 images belonging to 102 classes.


In [7]:
test_processing =  keras.preprocessing.image.ImageDataGenerator(
    rotation_range = 25,
    width_shift_range = .3,
    height_shift_range = .3,
    zoom_range = .3)


test_generator = test_processing.flow_from_directory('101_ObjectCategories_split/test', 
                                                       target_size = (224, 224),
                                                       color_mode="rgb",
                                                       batch_size = 32,
                                                       seed = 1,
                                                       class_mode= 'categorical',
                                                       shuffle = True)

Found 4544 images belonging to 102 classes.


# Modello pre addestrato ResNet50

In questa sezione viene applicato il processo di fine-tuning dato in input il modello pre-trainato (ResNet50) e successivamente viene addestrato tale modello sulle immagini importate precedentemente.
In particolare, verranno aggiunti i seguenti layer alla rete pre-trainata:
+ Flatten, in modo da ottenere un array mono-dimensionale
+ Fully Connected da 512 neuroni e funzione di attivazione reLu

+ Fully Connected da 256 neuroni e funzione di attivazione reLu

+ Output, con dimensione pari al numero delle classi (102 classi) e funzione di attivazione softmax

La funzione di loss scelta è crossentropy e la metrica di valutazione è l'accuracy.\
Si è scelto di utilizzare 10 epoche per l'addestramento.
Il batch size era stato impostato in precedenza a 32.

In [8]:
base_Inc = keras.applications.ResNet50(include_top = False,
                    weights = 'imagenet', 
                    pooling = 'avg',
                    input_shape = (224, 224, 3))

94781440/94765736 [==============================] - 1s 0us/step


In [9]:
for layer in base_Inc.layers:
    layer.trainable = False

In [10]:
x = base_Inc.output
x = keras.layers.Flatten()(x)                                       
x = keras.layers.Dense(512, activation = 'relu')(x)
x = keras.layers.Dense(256, activation = 'relu')(x) 
pred = keras.layers.Dense(102, activation = 'softmax')(x) 

In [11]:
new_Inc = keras.Model(inputs  = base_Inc.input, outputs = pred)

In [12]:
new_Inc.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [13]:
new_Inc.compile(loss = keras.losses.categorical_crossentropy,
            optimizer = keras.optimizers.RMSprop(),
            metrics = ['accuracy'])


new_Inc.fit(train_generator,
          epochs=10,
          validation_data= test_generator);

Epoch 1/10
144/144 [==============================] - 149s 951ms/step - loss: 2.1913 - accuracy: 0.5117 - val_loss: 1.1979 - val_accuracy: 0.6985
Epoch 2/10
144/144 [==============================] - 133s 927ms/step - loss: 0.8574 - accuracy: 0.7615 - val_loss: 1.3190 - val_accuracy: 0.6503
Epoch 3/10
144/144 [==============================] - 134s 934ms/step - loss: 0.6176 - accuracy: 0.8248 - val_loss: 0.9021 - val_accuracy: 0.7584
Epoch 4/10
144/144 [==============================] - 134s 934ms/step - loss: 0.4596 - accuracy: 0.8643 - val_loss: 0.9517 - val_accuracy: 0.7599
Epoch 5/10
144/144 [==============================] - 136s 946ms/step - loss: 0.4286 - accuracy: 0.8709 - val_loss: 0.8929 - val_accuracy: 0.7731
Epoch 6/10
144/144 [==============================] - 134s 936ms/step - loss: 0.3404 - accuracy: 0.8935 - val_loss: 0.9371 - val_accuracy: 0.7894
Epoch 7/10
144/144 [==============================] - 135s 937ms/step - loss: 0.3150 - accuracy: 0.9054 - val_loss: 0.9614 -

In [14]:
#Valuto le performance
new_Inc.evaluate(test_generator)

142/142 [==============================] - 67s 471ms/step - loss: 1.0702 - accuracy: 0.7912


[1.0702073574066162, 0.7911531925201416]

In [15]:
#salvataggio modello
new_Inc.save('/drive/MyDrive/model1.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


# Data augmentation (opzioni base)

Effettuo le stesse operazioni di prima senza modificare le opzioni di base della funzione di data aumentation

In [16]:
from tensorflow.keras.preprocessing import image as kimage

In [17]:
train_processing = kimage.ImageDataGenerator()

train_generator = train_processing.flow_from_directory('101_ObjectCategories_split/train', 
                                                       target_size = (224, 224),
                                                       color_mode="rgb",
                                                       batch_size = 32,
                                                       seed = 1,
                                                       class_mode= 'categorical',
                                                       shuffle = True)

Found 4600 images belonging to 102 classes.


In [18]:
test_processing = kimage.ImageDataGenerator()

test_generator = test_processing.flow_from_directory('101_ObjectCategories_split/test', 
                                                       target_size = (224, 224),
                                                       color_mode="rgb",
                                                       batch_size = 32,
                                                       seed = 1,
                                                       class_mode= 'categorical',
                                                       shuffle = True)

Found 4544 images belonging to 102 classes.


In [19]:
base_Inc2 = keras.applications.ResNet50(include_top = False,
                    weights = 'imagenet', 
                    pooling = 'avg',
                    input_shape = (224, 224, 3))

In [20]:
for layer in base_Inc2.layers:
    layer.trainable = False

In [21]:
x = base_Inc2.output
x = keras.layers.Flatten()(x)                                       
x = keras.layers.Dense(512, activation = 'relu')(x)
x = keras.layers.Dense(256, activation = 'relu')(x) 
pred = keras.layers.Dense(102, activation = 'softmax')(x) 

In [22]:
new_Inc2 = keras.Model(inputs  = base_Inc2.input, outputs = pred)

In [23]:
new_Inc2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [24]:
new_Inc2.compile(loss = keras.losses.categorical_crossentropy,
            optimizer = keras.optimizers.RMSprop(),
            metrics = ['accuracy'])


new_Inc2.fit(train_generator,
          epochs=10,
          validation_data= test_generator);

Epoch 1/10
144/144 [==============================] - 69s 454ms/step - loss: 1.7614 - accuracy: 0.5987 - val_loss: 0.8561 - val_accuracy: 0.7667
Epoch 2/10
144/144 [==============================] - 64s 444ms/step - loss: 0.5256 - accuracy: 0.8493 - val_loss: 0.6199 - val_accuracy: 0.8341
Epoch 3/10
144/144 [==============================] - 64s 444ms/step - loss: 0.2899 - accuracy: 0.9139 - val_loss: 0.6193 - val_accuracy: 0.8400
Epoch 4/10
144/144 [==============================] - 64s 445ms/step - loss: 0.1753 - accuracy: 0.9476 - val_loss: 1.2678 - val_accuracy: 0.7923
Epoch 5/10
144/144 [==============================] - 64s 445ms/step - loss: 0.1303 - accuracy: 0.9580 - val_loss: 0.9283 - val_accuracy: 0.8121
Epoch 6/10
144/144 [==============================] - 64s 444ms/step - loss: 0.1088 - accuracy: 0.9711 - val_loss: 0.6172 - val_accuracy: 0.8682
Epoch 7/10
144/144 [==============================] - 64s 444ms/step - loss: 0.0788 - accuracy: 0.9809 - val_loss: 0.8178 - val_ac

In [25]:
new_Inc2.evaluate(test_generator)

142/142 [==============================] - 32s 225ms/step - loss: 1.0706 - accuracy: 0.8303


[1.0706348419189453, 0.830325722694397]

# Conclusioni

I risultati ottenuti sono migliori utilizzando la seconda tipologia di data augmentation, lasciando invariata la struttura della rete, il numero di parametri addestrabili, batch size ed epoche. 
Infatti con i primi valori di data augmentation si arriva ad ottenere un valore di accuracy pari a 79% mentre con la seconda configurazione l'accuracy arriva fino all'83%.
